In [ ]:
import os
import subprocess
import shutil
import sys
##install openbabel and mopac
def run_command(command, error_message):
    print(f"Running: {command}")
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"{error_message}")
        print(result.stderr)
        exit(1)
    return result.stdout

def is_installed(command_name):
    return shutil.which(command_name) is not None

def install_openbabel():
    print("🔍 Checking Open Babel installation...")
    if not is_installed("obabel"):
        print("Installing Open Babel...")
        run_command("sudo apt-get update && sudo apt-get install -y openbabel", "Failed to install Open Babel.")
        print("Open Babel is installed.")
    else:
        print("Open Babel is already installed.")

def check_mopac():
    print("🔍 Checking MOPAC installation...")
    if not is_installed("mopac"):
        print("""
MOPAC is not installed.

Manual installation required:
1. Go to: https://openmopac.net/downloads.html
2. Download the appropriate version
3. Extract the files and move the 'MOPAC' binary to /usr/local/bin or a directory in your PATH.

Instuction:
$ tar -xvf MOPAC202X_Linux.tar.gz
$ sudo mv MOPAC2016.exe /usr/local/bin/mopac
$ sudo chmod +x /usr/local/bin/mopac

Then re-run this script.
""")
        exit(1)
    else:
        print("MOPAC is installed.")


In [15]:
def parse_total_charge_from_mol2(mol2_file):
    total_charge = 0.0
    with open(mol2_file, 'r') as f:
        in_atom_section = False
        for line in f:
            if line.startswith("@<TRIPOS>ATOM"):
                in_atom_section = True
                continue
            elif line.startswith("@<TRIPOS>"):
                in_atom_section = False
            elif in_atom_section:
                try:
                    fields = line.strip().split()
                    charge = float(fields[-1])
                    total_charge += charge
                except ValueError:
                    continue
    return round(total_charge)
def edit_mop_file(input_file, output_file, charge):
    with open(input_file, 'r') as f:
        lines = f.readlines()
    lines[0] = f"PM7 PREC DENSITY MULLIK EF GNORM=0.001 LET PRECISE CHARGE={charge}\n"
    with open(output_file, 'w') as f:
        f.writelines(lines)
    with open(input_arc, 'r') as f:
        lines = f.readlines()
    for i, line in enumerate(lines):
        if "Empirical Formula" in line:
            lines = lines[i:]
            break
    lines[0] = "PM7 PREC DENSITY MULLIK EF GNORM=0.001 LET PRECISE CHARGE=0\n"
    with open(output_mop, 'w') as f:
        f.writelines(lines)

In [18]:
def main():
    install_openbabel()
    check_mopac()

    ligand_pdb = "ligand_marvin.pdb"
    if not os.path.exists(ligand_pdb):
        print(f"Input file '{ligand_pdb}' not found in current directory.")
        exit(1)

    try:
        total_charge = int(input("Enter total charge of the ligand (integer): "))
    except ValueError:
        print("Invalid input. Please enter an integer.")
        exit(1)

    mol2_file = "ligand_h.mol2"
    run_command(f"obabel -ipdb {ligand_pdb} -omol2 -O {mol2_file}",
                "Failed to convert PDB to MOL2.")

    mol2_charge = parse_total_charge_from_mol2(mol2_file)
    if mol2_charge != total_charge:
        print(f"Charge mismatch! MOL2 sum = {mol2_charge}, Expected = {total_charge}")
        exit(1)
    else:
        print(f"Charge check passed: {mol2_charge}")

    minimized_sd = "ligand_h_sd.mol2"
    minimized_cg = "ligand_h_cg.mol2"
    run_command(f"obminimize -omol2 -ff mmff94 -n 100000 -sd {mol2_file} > {minimized_sd}",
                "Open Babel steepest descent minimization failed.")
    run_command(f"obminimize -omol2 -ff mmff94 -n 10000 -c 1e-7 -cg {minimized_sd} > {minimized_cg}",
                "Open Babel conjugate gradient minimization failed.")

    mop_file = "ligand_h.mop"
    run_command(f"obabel -imol2 {minimized_cg} -omop -O {mop_file}",
                "Failed to convert MOL2 to MOP.")

    edit_mop_file(mop_file, mop_file, total_charge)
    run_command(f"mopac {mop_file}", "MOPAC energy minimization failed.")

    arc_file = "ligand_h.arc"
    final_mop = "ligand_F.mop"
    if not os.path.exists(arc_file):
        print("ligand_h.arc not found after MOPAC run.")
        exit(1)
    clean_arc_to_mop(arc_file, final_mop)

    final_pdb = "ligand_opt.pdb"
    run_command(f"obabel -imop {final_mop} -opdb -O {final_pdb}",
                "Failed to convert final MOP to PDB.")

    print("\nLigand preparation completed successfully.")
    print(f"Final optimized ligand PDB: {final_pdb}")